In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lineagevi as linvi
import scanpy as sc
import scvelo as scv
import os

In [3]:
raw_path = '/Users/lgolinelli/git/lineageVI/notebooks/data/inputs/anndata/raw'
dataset_name = 'pancreas'
raw_adata_path = os.path.join(raw_path, dataset_name + '.h5ad')
os.makedirs(raw_path, exist_ok=True)
adata = scv.datasets.pancreas(raw_adata_path)

In [4]:
sc.pp.pca(adata, n_comps=100)
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000, subset_highly_variable=True, log=True)
scv.pp.neighbors(adata, n_neighbors=200)
scv.pp.moments(adata, n_neighbors=200)

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=20 + 1, metric='euclidean', n_jobs=-1)
nbrs.fit(adata.obsm['X_pca'])
distances, indices = nbrs.kneighbors(adata.obsm['X_pca'])

adata.uns['indices'] = indices

Filtered out 20801 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors


/Users/lgolinelli/miniconda3/envs/test1/lib/python3.11/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)
/var/folders/1x/8dphwyvs0j504xrq138nfssh0000gn/T/ipykernel_14153/3504472723.py:3: DeprecationWarning: `neighbors` is deprecated since scvelo==0.4.0 and will be removed in a future version of scVelo. Please compute neighbors with Scanpy.
  scv.pp.neighbors(adata, n_neighbors=200)


    finished (0:00:01) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [5]:
annotation_path = '/Users/lgolinelli/git/lineageVI/notebooks/data/inputs/gene_sets/'
annotation_name = 'msigdb_development_or_pancreas.gmt'
file_path = os.path.join(annotation_path, annotation_name)
os.makedirs(annotation_path, exist_ok=True)

linvi.utils.add_annotations(
    adata, 
    files=[file_path],
    min_genes=12,
    varm_key='I',
    uns_key='terms',
    clean=True,
    genes_use_upper=True)

adata._inplace_subset_var(adata.varm['I'].sum(1) > 0)

In [6]:
processed_dir_path = '/Users/lgolinelli/git/lineageVI/notebooks/data/inputs/anndata/processed/'
processed_adata_path = os.path.join(processed_dir_path, dataset_name + '.h5ad')
os.makedirs(processed_dir_path, exist_ok=True)
adata.write_h5ad(processed_adata_path)

"processed_dir_path = '/Users/lgolinelli/git/lineageVI/notebooks/data/inputs/anndata/processed/'\nprocessed_adata_path = os.path.join(processed_dir_path, dataset_name + '.h5ad')\nos.makedirs(processed_dir_path, exist_ok=True)\nadata.write_h5ad(processed_adata_path)"